Basic Imports


In [ ]:
# basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# for utilising GPU
import cupy as cp

### Loading the dataset

In [ ]:
train_df = pd.read_csv('/content/MNIST_train.csv')
test_df = pd.read_csv('/content/MNIST_test.csv')

In [ ]:
# we dont need so many index so dropping the columns:
train_df = train_df.drop(['Unnamed: 0', 'index'], axis=1)
test_df = test_df.drop(['Unnamed: 0', 'index'], axis=1)

In [ ]:
# convert data to the range of 0 to 1
scaled_train_X = (train_df.drop('labels', axis=1)) / 255.0
scaled_test_X = (test_df.drop('labels', axis=1)) / 255.0

### Seperating our dataset into features and target variable

In [ ]:
# Separate features (X) and target variable (y)
X_train = scaled_train_X
y_train = train_df['labels']

In [ ]:
# same for test set
X_test = scaled_test_X
y_test = test_df['labels']

### Converting the shape of X and y in the required numpy array

In [ ]:
# changing the shape of the data for the model
y_train = y_train.to_numpy()
X_train = X_train.to_numpy()
# same for the test set
y_test = y_test.to_numpy()
X_test = X_test.to_numpy()

### Splitting the train data into train and validation set to get the best hyperparameters

In [ ]:
# split the training data into training and validation sets
split_ratio = 0.8  # 80% of data for training, 20% for validation
split_index = int(len(X_train) * split_ratio)

X_train_split = X_train[:split_index]
y_train_split = y_train[:split_index]

X_val_split = X_train[split_index:]
y_val_split = y_train[split_index:]

## KNN CLASSIFIER

In [ ]:
# KNN classifier
class KNNClassifier:
    def fit(self, X, y):
        self.X = cp.array(X)
        self.y = cp.array(y)

    def predict(self, X, K, epsilon=1e-3):
        X = cp.array(X)
        N = len(X)
        y_hat = cp.zeros(N)
        for i in range(N):
            dist2 = cp.sum((self.X - X[i])**2, axis=1)
            idxt = cp.argsort(dist2)[:K]
            gamma_k = 1 / (cp.sqrt(dist2[idxt] + epsilon))
            y_hat[i] = cp.bincount(self.y[idxt], weights=gamma_k).argmax()

        return cp.asnumpy(y_hat)  # Convert back to NumPy array for compatibility with other code

Trying KNN with different hyperparameters

In [ ]:
# function to evaluate the model with given hyperparameters
def evaluate_model(X_train, y_train, X_val, y_val, K, epsilon):
    model = KNNClassifier()
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val, K, epsilon)
    accuracy = np.mean(y_val_pred == y_val)
    return accuracy

In [ ]:
# define the hyperparameter values
K_values = [3, 5, 7, 10 , 15, 20]
epsilon_values = [0.6, 0.01]

best_accuracy = 0
best_K = 0
best_epsilon = 0

# checking for all combinations of K and epsilon
for K in K_values:
    for epsilon in epsilon_values:
        accuracy = evaluate_model(X_train_split, y_train_split, X_val_split, y_val_split, K, epsilon)
        print(f"K: {K}, Epsilon: {epsilon}, Accuracy: {accuracy}")
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_K = K
            best_epsilon = epsilon

print(f"Best Hyperparameters - K: {best_K}, Epsilon: {best_epsilon}, Accuracy: {best_accuracy}")

K: 3, Epsilon: 0.6, Accuracy: 0.9713333333333334
K: 3, Epsilon: 0.01, Accuracy: 0.9713333333333334
K: 5, Epsilon: 0.6, Accuracy: 0.9705833333333334
K: 5, Epsilon: 0.01, Accuracy: 0.9705833333333334
K: 7, Epsilon: 0.6, Accuracy: 0.9690833333333333
K: 7, Epsilon: 0.01, Accuracy: 0.9690833333333333
K: 10, Epsilon: 0.6, Accuracy: 0.9685
K: 10, Epsilon: 0.01, Accuracy: 0.9685
K: 15, Epsilon: 0.6, Accuracy: 0.9646666666666667
K: 15, Epsilon: 0.01, Accuracy: 0.9645833333333333
K: 20, Epsilon: 0.6, Accuracy: 0.96225
K: 20, Epsilon: 0.01, Accuracy: 0.96225
Best Hyperparameters - K: 3, Epsilon: 0.6, Accuracy: 0.9713333333333334


1 hr 7 min to execute

Trying a few more combinations

In [ ]:
# define the hyperparameter values
K_values = [1,2]
epsilon_values = [ 0.01]

best_accuracy = 0
best_K = 0
best_epsilon = 0

# checking for all combinations of K and epsilon
for K in K_values:
    for epsilon in epsilon_values:
        accuracy = evaluate_model(X_train_split, y_train_split, X_val_split, y_val_split, K, epsilon)
        print(f"K: {K}, Epsilon: {epsilon}, Accuracy: {accuracy}")
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_K = K
            best_epsilon = epsilon

print(f"Hyperparameters - K: {best_K}, Epsilon: {best_epsilon}, Accuracy: {best_accuracy}")

K: 1, Epsilon: 0.01, Accuracy: 0.9681666666666666
K: 2, Epsilon: 0.01, Accuracy: 0.9681666666666666
Hyperparameters - K: 1, Epsilon: 0.01, Accuracy: 0.9681666666666666


time taken: 11 min

highest accuracy acheived is 97 %

___________